# Problem solvers
This notebook aims to show some interfaces useful to implement an algorithm that solves a particular problem. We present 3 cases in increase order of complexity.

## The simple function
The simplest way to implement a solver is to use a simple function such as

In [4]:
def solve_problem(X, Y, a=1, b=None):
    """This function solves problem P by using algorithm A.
    
        Args:
            X: ...
            Y: ...
            a: ...
            b: ...
        
        Returns:
            result: ...
        """
    
    # Preprocessing
    # ...
    
    # Algorithm
    for i in range(a):
        # do stuff
        pass
    
    # Postprocessing
    # ...
    result = b or Y
    
    return result

In [5]:
solve_problem(1,2)

2

In [6]:
solve_problem(1,2,b='b')

'b'

This method has advantages and disadvantages, for examples:

Advantages:
- Architecture simple to understand.
- Quick to implement.

Disadvantages:
- Can be hard to read if there are several parameters and options and hundreds of lines of code.
- Not flexible: Adding a variant to the algorithm implies either copy the whole function (very bad) to change some lines, or to add parameters and cases, which impairs readability.
- "Burnable": Once executed, the only thing left is the result. There is no trace of the algorithm, nor metadata, etc.
- No API: The solver does not provide ways to handle the result, such as saving or loading data, and treating or vizualizing the results.

When to use it:
- The algorithm is _short_ and _basic_: straightforward procedure, limited pre- and postprocessing.
- The problem is _narrow_: Not many parameters, no variant of the algorithm.
- The returned result is _simple_ and is a _standalone_: It is an instance of a base type and easy to handle.

It can be useful as a first implementation of something needed now, but that will eventually be changed to satisfy more need. A hint that this approach is not the right model is that the function is too long, or that is uses many subroutines that are defined exclusively for this solver.

## The single class
The next to simplest case is the use of a class. This model is very more flexible, but sometime at the cost of some clarity. Let us take a look at a template:

In [5]:
class ProblemSolver:
    """
    This class solves problem P with algorithm A.
    It provides a lot of flexibility via parameters and the constructor.
    It also implements an API to easily handle the results and the model.
    """
    def __init__(self, param_specific_to_the_algorithm):
        """
        Some doc...
        
        Parameters passed here should be about the algorithm, not about the problem to solve.
        """
        self.param_specific_to_the_algorithm = param_specific_to_the_algorithm
        
        # Preprocessing part of the algorithm
        # ...
    
    def solve(self, param_specific_to_the_instance_of_the_problem):
        """
        Explanations about the problem, algorithm and parameters.
        
        Parameters passed here should concern only the problem at hand, not the algorithm.
        """
        
        data = self.preprocess_data(param_specific_to_the_instance_of_the_problem)
        
        for i in range(1):
            # do stuff to data
            data = self.solve_part1(data)
            data = self.solve_part2(data)
            result = data
            pass
        
        # Note how we can attach the result to the solver itself so that it can carry many more things.
        self.result = self.postprocess_result(result, param_specific_to_the_instance_of_the_problem)
        # However, note that we did not attach data to the class since it is not useful once the algorithm is completed (keep in mind the lifespan of the variables).
        
        return self # Returns self for convenience.
    
    def preprocess_data(self, param):
        """
        This preprocessing function should be omitted if it is very simple and obvious.
        """
        # do stuff
        return param
    
    def postprocess_result(self, result, param):
        """
        This postprocessing function should be omitted if it is very simple and obvious.
        """
        # do stuff
        return param
    
    def solve_part1(self, data):
        """
        Computes part 1 of algorithm.
        """
        return data

    def solve_part2(self, data):
        """
        Computes part 2 of algorithm.
        """
        return data
    
    # API functions
    def save(self, filename):
        # save self to file
        pass
    
    @staticmethod # The 'staticmethod' decorator makes that the method does not need to use 'self'. The method can them be used without instanciating the class.
    def load(filename):
        """
        Should look like:
        
        loaded_model = pickle.load(filename)
        return loaded_model
        """
        pass
    
    def vizualize(self):
        """
        Do something with results and saved parameters.
        """
        print(self.result)
    
    # Other API implementations are data model functions (i.e. __methods__)
    def __len__(self):
        return len(self.result)
    
    def __iter__(self):
        yield from self.result
    
    def __str__(self):
        return str(self.param_specific_to_the_algorithm)

In [6]:
solver = ProblemSolver('algo params').solve('my problem')
solver.vizualize()

my problem


Let us take a look at advantages and disadvantages.

Advantages:
- Easy to understand each part individually.
- Customization of the algorithm is separated from the specific problem (i.e. constructor initialization vs solve method).
- Metadata and important parameters/information can be stored in the solver alongside with the result.
- API is integrated directly in the solver, making it easy to handle the result, but also the solver itself.
- Major algorithm variants can be implemented via inheritance instead of a parameter in the constructor for more clarity.
- Easy to extend as it is a class.

Disadvantages:
- Quite involved to write.
- Sometime harder to understand the mechanics as a whole when things become convoluted.

When to use it:
- The result needs special ways to be processed and handled.
- Metadata and information about the procedure are important.
- The algorihtm and the problem takes many parameters and come in many variants.
- The algorithm is _not long_ and/or _not very complex_.

Be aware that the class should not store unecessary attributes. For example, you do not want to store the problem parameters  if they are not useful once the algorithm is completed because they take space in memory and pollute the namespace.

This approach turns out to be very powerful. However, sometime the algorithm is long and complex and the class becomes very long and hard to understand. In that case, it is better to uses the next method I propose.

## The API-Algorithm-Result class scheme
When the class becomes too burden, it is generally a good idea to split it in three parts: the API, the Algorithm and the Result. This aims to increase greatly readability and clarity. It goes as follow.

In [7]:
class ProblemSolver:
    """
    This is the API: the interface between user and the algorithm.
    It does not actually solves the problem; it delegates it to an other class not meant to be used by a user.
    However, it does provides methods to easily work with the algorithm.
    
    The implementation should be identical to the one written above, with the exception of the 'solve' method
    and that the 'solve_part1' and 'solve_part2' that are not present.
    """
    def solve(self, param_specific_to_the_instance_of_the_problem):
        algorithm = Algorithm(self.param_specific_to_the_algorithm, param_specific_to_the_instance_of_the_problem)
        self.result = algorithm.solve()
        
        return self
    
class Algorithm:
    """
    This class implements algorithm A. Should be used through ProblemSolver.
    """
    def __init__(self, algo_param, problem_param):
        self.algo_param = algo_param
        self.problem_param = problem_param # Note that we can save the parameters of the problem inside the class since the class has a limited lifespan.
        # Preprocessing
        # ...
    
    def solve(self):
        
        for i in range(1):
            # do stuff
            # Note that we do not have to pass parameters anymore since they can be accessed throught the attributes of the class.
            self.solve_part1()
            result = self.solve_part2()
            pass
        
        self.result_part1 = result.result_part1
        self.result_part2 = result.result_part2
        
        return self
            
    def solve_part1(self):
        """
        Computes part 1 of algorithm.
        """
        # Do stuff to self.problem_param
        self.problem_param += 1

    def solve_part2(self):
        """
        Computes part 2 of algorithm.
        """
        return Result(self.problem_param)
        

class Result:
    """
    This class implements a composite result with many attributes.
    
    It facilitates the transfer of a result from the algorithm to the API.
    It can act as an augmented dictionary (or list).
    It can also provides basic method to handle the data, like packing and unpacking.
    It is not always useful to implement this object.
    It is appropriate to use properties in this kind of object.
    """
    def __init__(self, result_part1, result_part2):
        self.result_part1 = result_part1
        self.result_part2 = result_part2
    
    def update(self, result_part1, result_part2):
        """
        For example, update only if result_part is better than previous.
        """
        if result_part1 > self.result_part1:
            self.result_part1 = result_part1
            
        if result_part2 > self.result_part2:
            self.result_part2 = result_part2
        

Advantages:
- Very modular implementation: easy to understand each role and to modify any part.
- Very flexible.
- All advantages of the single class.

Disadvantages:
- Can be hard to understand as a whole.
- Very involved to write.

When to use it:
- The algorithm is long and complex.
- All other cases of the single class.

Hints you should opt for this scheme:
- The algorithm is splitted in many sub parts.
- Theses subparts uses the same parameters: creating a class and storing parameters as attributes can make it easy to them and can increase clarity. A good hint is to keep in mind the lifespan of your variables. 